In [1]:
import pandas as pd
import numpy as np
from tqdm import tqdm

In [2]:
def na_table(x):
    na = pd.DataFrame(x.isna().sum())
    nna = pd.DataFrame(x.notna().sum())
    na.reset_index(inplace=True)
    nna.reset_index(inplace=True)
    na = na[na[0]>0]
    na = na.merge(nna, on='index')
    na.columns = ['col', 'isNA', 'notNA']
    return na

In [3]:
data_dir = '../train/'
features_dir = '../features/'
annual_report_info = pd.read_csv(data_dir + 'annual_report_info.csv', encoding='gbk')
annual_report_info.drop_duplicates(inplace=True)
a = annual_report_info.copy()
na = na_table(annual_report_info)
na

,col,isNA,notNA
0,STATE,5,22443
1,FUNDAM,16749,5699
2,MEMNUM,22419,29
3,FARNUM,22419,29
4,ANNNEWMEMNUM,22419,29
5,ANNREDMEMNUM,22419,29
6,EMPNUM,15,22433
7,EMPNUMSIGN,5714,16734
8,BUSSTNAME,4870,17578
9,COLGRANUM,2488,19960


In [4]:
annual_report_info['STATE'].fillna(0, inplace=True)
# annual_report_info['FUNDAM'].fillna(annual_report_info['FUNDAM'].mean(),inplace=True)
# annual_report_info['EMPNUM'].fillna(annual_report_info['EMPNUM'].mean(),inplace=True)
annual_report_info['EMPNUMSIGN'].fillna(0, inplace = True)
# annual_report_info['BUSSTNAME'].fillna('Unknown',inplace=True)
# annual_report_info['UNEEMPLNUM'].fillna('Unknown', inplace = True)
# annual_report_info['DISEMPLNUM'].fillna('Unknown', inplace = True)
# annual_report_info['RETEMPLNUM'].fillna('Unknown', inplace = True)
# annual_report_info['COLEMPLNUM'].fillna('Unknown', inplace = True)
# annual_report_info['UNENUM'].fillna('Unknown', inplace = True)
# annual_report_info['DISPERNUM'].fillna('Unknown', inplace = True)
# annual_report_info['RETSOLNUM'].fillna('Unknown', inplace = True)
# annual_report_info['COLGRANUM'].fillna('Unknown', inplace = True)
annual_report_info['WEBSITSIGN'].fillna(0, inplace = True)
annual_report_info['FORINVESTSIGN'].fillna(0, inplace = True)
annual_report_info['STOCKTRANSIGN'].fillna(0, inplace = True)
annual_report_info['PUBSTATE'].fillna(0, inplace = True)
na_table(annual_report_info)

,col,isNA,notNA
0,FUNDAM,16749,5699
1,MEMNUM,22419,29
2,FARNUM,22419,29
3,ANNNEWMEMNUM,22419,29
4,ANNREDMEMNUM,22419,29
5,EMPNUM,15,22433
6,BUSSTNAME,4870,17578
7,COLGRANUM,2488,19960
8,RETSOLNUM,2488,19960
9,DISPERNUM,2488,19960


In [5]:
num_cols = [i for i in annual_report_info.columns if i[-1]=='M']
cat_cols = [i for i in annual_report_info.columns if i[-2:]=='GN' or i[-2:]=='TE']
grouped = annual_report_info.groupby('id')
annual_report_info = annual_report_info.groupby('id').count().reset_index()
for i in tqdm(annual_report_info['id']):
    for j in num_cols:
        annual_report_info.loc[annual_report_info.id==i,j] = grouped.get_group(i)[j].fillna(0).diff().mean()
    for j in cat_cols:
        annual_report_info.loc[annual_report_info.id==i,j] = grouped.get_group(i)[j].mean()

100%|██████████| 8937/8937 [05:05<00:00, 29.21it/s]


In [6]:
annual_report_info.set_index(keys='id', inplace=True)
annual_report_info.to_csv(features_dir + 'annual_report_info.csv')

In [7]:
annual_report_info = a
annual_report_info.fillna(-1, inplace=True)
annual_report_info = annual_report_info.pivot_table(index='id', columns='ANCHEYEAR').fillna(0)
annual_report_info.replace(-1, pd.NA, inplace=True)
annual_report_info.head()

ANNNEWMEMNUM                \
ANCHEYEAR                                              2015.0 2016.0 2017.0   
id                                                                            
175ebe5f059ec05007223e9af0a48b885f4cbfa833d93eed          NaN    NaN    NaN   
175ebe5f059ec05036d901021be6da41057ae3ee1fe6b8bb          NaN    NaN    NaN   
175ebe5f059ec050efe07058fc35890a2b8858a6795a2e24          0.0    NaN    NaN   
216bd2aaf4d0792406c041069b786b3bcb5baa4cf80d5987          0.0    0.0    0.0   
216bd2aaf4d079240c2b8f7bbf3177618ad289f21af0221e          NaN    NaN    NaN   

                                                        ANNREDMEMNUM         \
ANCHEYEAR                                        2018.0       2015.0 2016.0   
id                                                                            
175ebe5f059ec05007223e9af0a48b885f4cbfa833d93eed    NaN          NaN    NaN   
175ebe5f059ec05036d901021be6da41057ae3ee1fe6b8bb    NaN          NaN    NaN   
175ebe5f059ec050efe07058fc35890a2b8858a6795a2e24    NaN          0.0    NaN   
216bd2aaf4d0792406c041069b786b3bcb5baa4cf80d5987    NaN          0.0    0.0   
216bd2aaf4d079240c2b8f7bbf3177618ad289f21af0221e    NaN          NaN    NaN   

                                                               COLEMPLNUM  \
ANCHEYEAR                                        2017.0 2018.0     2015.0   
id                                                                          
175ebe5f059ec05007223e9af0a48b885f4cbfa833d93eed    NaN    NaN        0.0   
175ebe5f059ec05036d901021be6da41057ae3ee1fe6b8bb    NaN    NaN       10.0   
175ebe5f059ec050efe07058fc35890a2b8858a6795a2e24    NaN    NaN        0.0   
216bd2aaf4d0792406c041069b786b3bcb5baa4cf80d5987    0.0    NaN        0.0   
216bd2aaf4d079240c2b8f7bbf3177618ad289f21af0221e    NaN    NaN        0.0   

                                                         ... UNEEMPLNUM  \
ANCHEYEAR                                        2016.0  ...     2017.0   
id                                                       ...              
175ebe5f059ec05007223e9af0a48b885f4cbfa833d93eed    0.0  ...        NaN   
175ebe5f059ec05036d901021be6da41057ae3ee1fe6b8bb   10.0  ...        0.0   
175ebe5f059ec050efe07058fc35890a2b8858a6795a2e24   25.0  ...        2.0   
216bd2aaf4d0792406c041069b786b3bcb5baa4cf80d5987    0.0  ...        0.0   
216bd2aaf4d079240c2b8f7bbf3177618ad289f21af0221e    0.0  ...        0.0   

                                                        UNENUM                \
ANCHEYEAR                                        2018.0 2015.0 2016.0 2017.0   
id                                                                             
175ebe5f059ec05007223e9af0a48b885f4cbfa833d93eed    NaN    0.0    0.0    NaN   
175ebe5f059ec05036d901021be6da41057ae3ee1fe6b8bb    0.0    0.0    0.0    0.0   
175ebe5f059ec050efe07058fc35890a2b8858a6795a2e24    0.0    0.0    0.0    0.0   
216bd2aaf4d0792406c041069b786b3bcb5baa4cf80d5987    0.0    0.0    0.0    0.0   
216bd2aaf4d079240c2b8f7bbf3177618ad289f21af0221e    0.0    0.0    0.0    0.0   

                                                        WEBSITSIGN         \
ANCHEYEAR                                        2018.0     2015.0 2016.0   
id                                                                          
175ebe5f059ec05007223e9af0a48b885f4cbfa833d93eed    NaN        2.0    2.0   
175ebe5f059ec05036d901021be6da41057ae3ee1fe6b8bb    0.0        1.0    1.0   
175ebe5f059ec050efe07058fc35890a2b8858a6795a2e24    0.0        0.0    2.0   
216bd2aaf4d0792406c041069b786b3bcb5baa4cf80d5987    0.0        0.0    0.0   
216bd2aaf4d079240c2b8f7bbf3177618ad289f21af0221e    0.0        2.0    2.0   

                                                                
ANCHEYEAR                                        2017.0 2018.0  
id                                                              
175ebe5f059ec05007223e9af0a48b885f4cbfa833d93eed    2.0    2.0  
175ebe5f059ec05036d901021be6da41057ae3ee1fe6b8b

In [8]:
annual_report_info.reset_index(inplace=True)
cols = ['id']
cols.extend([str(j[0]) + str(j[1])[3:4] for j in annual_report_info.columns[1:]])
annual_report_info.columns = cols
annual_report_info.head()

,id,ANNNEWMEMNUM5,ANNNEWMEMNUM6,ANNNEWMEMNUM7,ANNNEWMEMNUM8,ANNREDMEMNUM5,ANNREDMEMNUM6,ANNREDMEMNUM7,ANNREDMEMNUM8,COLEMPLNUM5,...,UNEEMPLNUM7,UNEEMPLNUM8,UNENUM5,UNENUM6,UNENUM7,UNENUM8,WEBSITSIGN5,WEBSITSIGN6,WEBSITSIGN7,WEBSITSIGN8
0,175ebe5f059ec05007223e9af0a48b885f4cbfa833d93eed,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,...,NaN,NaN,0.0,0.0,NaN,NaN,2.0,2.0,2.0,2.0
1,175ebe5f059ec05036d901021be6da41057ae3ee1fe6b8bb,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,10.0,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,1.0,1.0
2,175ebe5f059ec050efe07058fc35890a2b8858a6795a2e24,0.0,NaN,NaN,NaN,0.0,NaN,NaN,NaN,0.0,...,2.0,0.0,0.0,0.0,0.0,0.0,0.0,2.0,2.0,2.0
3,216bd2aaf4d0792406c041069b786b3bcb5baa4cf80d5987,0.0,0.0,0.0,NaN,0.0,0.0,0.0,NaN,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.0
4,216bd2aaf4d079240c2b8f7bbf3177618ad289f21af0221e,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,2.0,2.0,2.0,2.0


In [9]:
annual_report_info.set_index(keys='id', inplace=True)
annual_report_info.to_csv(features_dir + 'annual_report_info_.csv')